## Comma Dashboard 
Early stage rough draft

#### Imports & Set up

In [ ]:
import os
import time
# dashboard 
import ipywidgets as widgets
from IPython.display import display
# data pipeline
import cudf
from blazingsql import BlazingContext
# dataviz
import datashader as ds
from datashader import transfer_functions as tf
# import holoviews as hv
from colorcet import fire
# from holoviews import opts
# from holoviews import extension, Points, Scatter
# import holoviews.operation.datashader as hd
# from holoviews.element.tiles import EsriImagery
# from holoviews.operation.datashader import datashade
# # config holoviews
# extension('bokeh')
# pw = 450
# ph = 450
# opts.defaults(opts.Scatter(width=pw, height=ph, color='blue'), opts.RGB(width=pw, height=ph), opts.Curve(width=pw))

# start up BlazingSQL
bc = BlazingContext()

# tag path to taxi data
path = os.getcwd() + '/' + 'data/c2k19*'
# how's it look?
path

In [ ]:
from datashader.utils import lnglat_to_meters

#### Create Table

In [ ]:
%%time
# create base table
bc.create_table('comma', path, header=0)

# what's it look like? 
bc.sql('select * from comma').tail(3)

In [ ]:
conv = lnglat_to_meters(longitude=bc.sql('SELECT longitude FROM comma')['longitude'],
                        latitude=bc.sql('SELECT latitude FROM comma')['latitude'])

print(type(conv), len(conv), [len(c) for c in conv])

In [ ]:
gdf = cudf.DataFrame()
gdf['x'] = conv[0]
gdf['y'] = conv[1]
cudf.concat([bc.sql('SELECT longitude, latitude FROM comma'), gdf], axis=1)

In [ ]:
ggdf = cudf.concat([bc.sql('SELECT longitude FROM comma'), gdf], axis=1)
ggdf.tail(2)

In [ ]:
# execute query & lay out the canvas w/ dropoff locations 
agg = ds.Canvas().points(gdf, 'x', 'y')
# shade in the picture w/ fire & display
display(tf.set_background(tf.shade(agg, cmap=fire), "black"))

In [ ]:
# bc.sql(f'SELECT longitude, latitude FROM comma LEFT JOIN {ggdf} ON (comma.longitude = {ggdf}.longitude)')

#### Pull Value Options 

In [ ]:
# query to extract time values from pickup time
time_table = '''
             SELECT
                 DAYOFMONTH(cast(utc_timestamp as timestamp)) days,
                 MONTH(cast(utc_timestamp as timestamp)) months,
                 YEAR(cast(utc_timestamp as timestamp)) years
             FROM
                 comma
                 '''

# unique value finder
def unique_sorted_values(array):
    # identify unique values 
    unique = array.unique()
    # convert to list & sort the values
    unique.tolist().sort()
    # return sorted tuple (works best w/ widgets)
    return tuple(unique)

# make temp dataframe from time query + passenger_count 
temp_df = bc.sql(time_table.replace('SELECT', 'SELECT speed, altitude, bearing,'))

# tag day, month, year & passenger count values for widgets
dy_vals = unique_sorted_values(temp_df.days)
mo_vals = unique_sorted_values(temp_df.months)
yr_vals = unique_sorted_values(temp_df.years)
speed_vals = unique_sorted_values(temp_df.speed)
speed_vals = unique_sorted_values(temp_df.bearing)

# forget temp df
del temp_df

# dropdown days
dropdown_day = widgets.SelectMultiple(value=dy_vals, options=dy_vals, description='Days of Month')

# month options  
months = {1:'. Jan', 2:'. Feb', 3:'. Mar', 4:'. Apr', 5:'. May', 6:'. Jun', 
          7:'. Jul', 8:'. Aug', 9:'. Sep', 10:'. Oct', 11:'. Nov', 12:'. Dec'}
mo_vals = [f'{k}{months[k]}' for k in months if k in mo_vals]
dropdown_month = widgets.SelectMultiple(value=mo_vals, options=mo_vals, description='Months')

# year 
dropdown_year = widgets.SelectMultiple(value=yr_vals, options=yr_vals, description='Years')  

# number of riders
dropdown_speed = widgets.Text(value='', 
                              placeholder='e.g. > 30 for 30 mph, blank for all',
                              description='Speed:')
# fare amount 
dropdown_altitude = widgets.Text(value='',
                                 placeholder='e.g. < 60 for under 60, blank for all',
                                 description='Altitude:')
# location 
dropdown_bearing = widgets.Text(value='',
                                placeholder='e.g. = 300 for 300, blank for all',
                                description='Bearing:')

# tag default dropdown values that affect SQL query adjustment
default_day = dropdown_day.value
default_month = dropdown_month.value
default_year = dropdown_year.value

#### Create / Enable (dashboard) Widgets 

In [ ]:
# build outputs (each will have it's own tab)
output = widgets.Output()
plot_output = widgets.Output()
hv_output = widgets.Output()
hv_output2 = widgets.Output()

## Query Processing & Dashboard Output
This & below is what will actually be running each time you engage with the dashboard.

In [ ]:
def common_filtering(day, month, year, speed, altitude, bearing):
    """sql: day, month, year, n_riders, tol_cost
    visual dataframe: month, year, n_riders, tol_cost
    plot: month, year, n_riders, tol_cost, location"""
            
    # clear outputs (replace instead of stacking, basically)
    output.clear_output()
    plot_output.clear_output()
    hv_output.clear_output()
    hv_output2.clear_output()
    
    # tag pi & origin shift
    pi = 3.141592653589793
    o_shift = 2 * pi * 6378137 / 2

    # start building our SQL query 
    query = f'''
            SELECT 
                *,
                longitude * {o_shift} / 180 AS x, 
                LOG10(TAN(((90 + latitude) * {pi} / 360))) / {pi} / 180 * {o_shift} / 180 AS y
            FROM 
                comma'''

    # has user focused specific days?
    if day != default_day:
        # go through days selected
        for d in list(day):
            # are days already being focused in the query?
            if 'days' in query:
                # add OR to include this year
                query += f' OR days = {d}'
            # ok, is query at all developed?
            elif 'WHERE' in query:
                # add AND statement for year 
                query = query + f' AND days = {d}'
            # WHERE is not in query yet
            else:
                # add WHERE & month to query 
                query += f' WHERE days = {d}'
    # has user focused specific months?
    if month != default_month:
        # go through months selected
        for m in list(month):
            # extract month number
            m = m.split('.')[0]
            # are months already being focused in the query?
            if 'months' in query:
                # add OR to include this month
                query += f' OR months = {m}'
            # ok, is query at all developed?
            elif 'WHERE' in query:
                # add AND statement for month 
                query = query + f' AND months = {m}'
            # WHERE is not in query yet
            else:
                # add WHERE & month to query 
                query += f' WHERE months = {m}'
    # has user focused specific years?
    if year != default_year:
        # go through years selected
        for y in list(year):
            # are years already being focused in the query?
            if 'years' in query:
                # add OR to include this year
                query += f' OR years = {y}'
            # ok, is query at all developed?
            elif 'WHERE' in query:
                # add AND statement for year 
                query = query + f' AND years = {y}'
            # WHERE is not in query yet
            else:
                # add WHERE & month to query 
                query += f' WHERE years = {y}'
    # has user focused specific rider counts? 
    if speed != '':
        # check how developed SQL statement is
        if 'WHERE' in query:
            # add AND statement for year 
            query = query + f' AND speed {speed}'
        # WHERE is not in query yet
        else:
            # add WHERE & month to query 
            query += f' WHERE speed {speed}'
    # has user focused specific fare values? 
    if altitude != '':
        # check how developed SQL statement is
        if 'WHERE' in query:
            # add AND statement for select total fares
            query = query + f' AND altitude {altitude}'
        # WHERE is not in query yet
        else:
            # add WHERE & total fare to query 
            query += f' WHERE altitude {altitude}'

    # make final adjustments to query 
    query = query.replace('comma', f"({time_table.replace('SELECT', 'SELECT *,')})")
    
    # create table of results (overwrite if existing)
    bc.create_table('r', bc.sql(query))  

    # data exploration output  
    with output:
        # what's my final query?
        print(f'{query}\n')  
        # show us the end of the results
        display(bc.sql(query).tail(25))
        
    # nyc fire map 
    with plot_output:
        # execute query & lay out the canvas w/ dropoff locations 
        agg = ds.Canvas().points(bc.sql(query).to_pandas(), 'x', 'y')
        # shade in the picture w/ fire & display
        display(tf.set_background(tf.shade(agg, cmap=fire), "black"))
    
    # tip amount vs number of riders
    with hv_output:
        # make a scatter plot
        s = Scatter(bc.sql('select speed, bearing from r').to_pandas(), 'speed', 'bearing')
        ranged = s.redim.range(speed=(0, 60), bearing=(-0.5, 6.5))
        shaded = hd.spread(hd.datashade(ranged, x_sampling=0.15))
        labelled = shaded.redim.label(speed="Speed", bearing="Bearing")
        # let's see
        display(labelled)
        
    with hv_output2:
        # graph out coords 
#         points = Points(bc.sql('SELECT x, y FROM r').to_pandas(), ['x', 'y'])
        # plot comma coord points
#         comma_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=fire, width=900, height=480)
        # generate map 
        map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
        # let's see plot & map combo
#         display(map_tiles * comma_trips)
        display(map_tiles)  # temp fix

#### Event Handlers (Dashboard Inputs)

In [ ]:
def dropdown_day_eventhandler(change):
    common_filtering(change.new, dropdown_month.value, dropdown_year.value, dropdown_speed.value, 
                     dropdown_fare.value, dropdown_bearing.value)
    
def dropdown_month_eventhandler(change):
    common_filtering(dropdown_day.value, change.new, dropdown_year.value, dropdown_speed.value, 
                     dropdown_altitude.value, dropdown_bearing.value)
    
def dropdown_year_eventhandler(change):
    common_filtering(dropdown_day.value, dropdown_month.value, change.new, dropdown_speed.value, 
                     dropdown_altitude.value, dropdown_bearing.value)
    
def dropdown_speed_eventhandler(change):
    # adjust change to remove errors 
    change = change.new.strip().replace('"','').replace("'",'')
    # check change to make sure it works
    valid = True
    # is input blank 
    if change != '':
        # make sure there's a direction
        if change[0] not in '<>=':
            # query not ready yet
            valid = False
        # make sure there's a number
        if change[-1] not in '1234567890':
            # query not ready yet
            valid = False
    # is the query ready?
    if valid == True:
        # run it
        common_filtering(dropdown_day.value, dropdown_month.value, dropdown_year.value, 
                         change, dropdown_altitude.value, dropdown_bearing.value)    
        
def dropdown_altitude_eventhandler(change):
    # adjust change to remove errors 
    change = change.new.strip().replace('"','').replace("'",'')
    # check change to make sure it works
    valid = True
    # is input blank 
    if change != '':
        # make sure there's a direction
        if change[0] not in '<>=':
            # query not ready yet
            valid = False
        # make sure there's a number
        if change[-1] not in '1234567890':
            # query not ready yet
            valid = False
    # is the query ready?
    if valid == True:
        # run it
        common_filtering(dropdown_day.value, dropdown_month.value, dropdown_year.value, 
                         dropdown_speed.value, change, dropdown_bearing.value) 
        
def dropdown_bearing_eventhandler(change):
    # adjust change to remove errors 
    change = change.new.strip().replace('"','').replace("'",'')
    # check change to make sure it works
    valid = True
    # is input blank 
    if change != '':
        # make sure there's a direction
        if change[0] not in '<>=':
            # query not ready yet
            valid = False
        # make sure there's a number
        if change[-1] not in '1234567890':
            # query not ready yet
            valid = False
    # is the query ready?
    if valid == True:
        # run it
        common_filtering(dropdown_day.value, dropdown_month.value, dropdown_year.value, 
                         dropdown_speed.value, dropdown_altitude.value, change) 
    
# We bind the handlers to the dropdowns, and that’s it!
dropdown_day.observe(dropdown_day_eventhandler, names='value')
dropdown_month.observe(dropdown_month_eventhandler, names='value')
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_speed.observe(dropdown_speed_eventhandler, names='value')
dropdown_altitude.observe(dropdown_altitude_eventhandler, names='value')
dropdown_bearing.observe(dropdown_bearing_eventhandler, names='value')

#### Define Input & Output Dashboards

In [ ]:
def input_dashboard():
    # The HBox will add widgets to it one at a time from left-to-right:
    input_widgets = widgets.HBox([dropdown_day, dropdown_month, dropdown_year])
    input_widgets1 = widgets.HBox([dropdown_speed, dropdown_altitude, dropdown_bearing]) 
    # dashboard inputs
    display(input_widgets)
    display(input_widgets1)
    
def display_dashboard():
    # dashboard outputs
    tab = widgets.Tab([output, plot_output, hv_output, hv_output2])
    tab.set_title(0, 'Dataset Exploration')
    tab.set_title(1, 'Coordinate Map')
    tab.set_title(2, 'Speed vs Bearing')
    tab.set_title(3, 'Geo Map')
    # display outputs 
    display(tab)

# Dashboard 

In [ ]:
input_dashboard()

In [ ]:
display_dashboard()